# Movies Scraping from TMDB and IMDB
## Team MovieGnat

**Here are the steps to take:**

1. Run Part 1 to grab all movies and keywords from TMDB API and put them into your CSV file titled: *tmdb-movies-1-to-400.csv*, *tmdb-movies-401-to-800.csv*, etc.

2. Run Part 2 to grab all the IMDB ids from the TMDB ids provided as the .csv file you generated in Step 1. Write the output into another CSV titled: *imdb-ids-1-to-400.csv*, *imdb-ids-401-to-800.csv*, etc.

3. Run Part 3 to grab features from IMDB API given IMDB ids provided as the .csv file generated in Step 2. Write the output into another CSV titles: *imdb-features-1-to-400.csv*, *imdb-features-401-to-800.csv*, etc.

4. (Not yet done) We will have to map the IMDB features back to the part 1 dataset of TMDB features based on ids.

In [4]:
# PART 1:  STANDALONE TO GRAB ALL MOVIES AND KEYWORDS

import csv
import time
import requests


#########################################################
'''
BASE STUFF THAT IS ALSO DEFINED ON TOP
'''
def requestResults(url):
    r = requests.get(BASE_URL + url + "&api_key=" + API_KEY)
    return r.json()

# Constants
BASE_URL = "https://api.themoviedb.org/3/"
API_KEY = "9767d17413ec9d9729c2cca238df02da"
GENRE_MAP = {}
for g in requestResults("genre/movie/list?x=1")[u'genres']:
    GENRE_MAP[g['id']] = g['name']
    
#########################################################


def _getKeywordsStringById(movie_id):
    
    keywords_dict = requestResults("movie/" + str(movie_id) + "/keywords?language=en-US")
    if u'keywords' not in keywords_dict:
        return ''
    keywords_dict = keywords_dict[u'keywords']
    kstring = ''
    for k in keywords_dict:
        kstring += k[u'name'] + ','
    return str(kstring.encode('utf-8').strip())[:-1]

def _tidyRow(m, keywords):
    # Makes sure the row of movie is well-formatted
    output = {}
    for k in m:
        typem = type(m[k])
        k = str(k)
        if typem == str or typem == unicode:
            output[k] = m[k].encode('utf-8').strip()
        else:
            output[k] = m[k]
    output['keywords'] = keywords
    return output

def downloadMoviesToCSV(start_page, increment, filename):
    genre_count = {}
    
    with open(filename, 'w') as csvfile:
        fieldnames = ['id', 'genre_ids', 'poster_path', 'title', 'overview', 'release_date', 
                      'popularity', 'original_title', 'backdrop_path', 'keywords', 
                     'vote_count', 'video', 'adult', 'vote_average', 'original_language']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        # Get keywords for movies
        writer.writeheader()
        
        # TMDB limits 4 requests per second
        hit = 3 # Once hit reaches 0, call timer and reset hit to 3
        
        for p in range(start_page,start_page+increment): 
            results_p = requestResults("discover/movie?sort_by=popularity.desc&page=" + str(p))[u'results']
            hit -= 1
            if hit <= 0:
                hit = 3
                time.sleep(1)

            # Write to CSV
            for m in results_p:
                mid = m[u'id']
                keywords = _getKeywordsStringById(mid)
                hit -= 1
                if hit <= 0:
                    hit = 3
                    time.sleep(1)
                
                row = _tidyRow(m, keywords)
                writer.writerow(row)
            print('%d pages done' % p)

In [5]:
### Run Part 1: REMEMBER TO CHANGE start_page to your start page, don't have to change increment
downloadMoviesToCSV(start_page=1, increment=400, filename='tmdb-movies-1-to-400.csv')

1 pages done
2 pages done
3 pages done
4 pages done
5 pages done
6 pages done
7 pages done
8 pages done
9 pages done
10 pages done
11 pages done
12 pages done
13 pages done
14 pages done
15 pages done
16 pages done
17 pages done
18 pages done
19 pages done
20 pages done
21 pages done
22 pages done
23 pages done
24 pages done
25 pages done
26 pages done
27 pages done
28 pages done
29 pages done
30 pages done
31 pages done
32 pages done
33 pages done
34 pages done
35 pages done
36 pages done
37 pages done
38 pages done
39 pages done
40 pages done
41 pages done
42 pages done
43 pages done
44 pages done
45 pages done
46 pages done
47 pages done
48 pages done
49 pages done
50 pages done
51 pages done
52 pages done
53 pages done
54 pages done
55 pages done
56 pages done
57 pages done
58 pages done
59 pages done
60 pages done
61 pages done
62 pages done
63 pages done
64 pages done
65 pages done
66 pages done
67 pages done
68 pages done
69 pages done
70 pages done
71 pages done
72 pages done
7

In [3]:
# PART 2: STANDALONE THAT TAKES IN .CSV FILE AND GETS ALL IMDB IDS in a separate file

import pandas as pd
import csv
import time
import requests


#########################################################
'''
BASE STUFF THAT IS ALSO DEFINED ON TOP
'''
def requestResults(url):
    r = requests.get(BASE_URL + url + "&api_key=" + API_KEY)
    return r.json()

# Constants
BASE_URL = "https://api.themoviedb.org/3/"
API_KEY = "9767d17413ec9d9729c2cca238df02da"
GENRE_MAP = {}
for g in requestResults("genre/movie/list?x=1")[u'genres']:
    GENRE_MAP[g['id']] = g['name']
    
#########################################################

def downloadIMDBIds(input_filename, output_filename):
    df = pd.read_csv(input_filename)

    with open(output_filename, 'w') as csvfile:
        fieldnames = ['id', 'imdb_id']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        # TMDB limits 4 requests per second
        hit = 3 # Once hit reaches 0, call timer and reset hit to 3

        count = 0
        for tmid in df['id']:
            count += 1
            results = requestResults('movie/' + str(tmid) + '?x=1')
            if u'imdb_id' not in results or results[u'imdb_id'] is None:
                continue
            imid = results[u'imdb_id'].strip('tt')
            row = {'id': tmid, 'imdb_id': imid}
            writer.writerow(row)
            hit -= 1
            if hit <= 0:
                hit = 3
                time.sleep(1)
            if count % 200 == 0:
                print 'done with %d movies' % count

In [4]:
### Run Part 2: Get imdb ids from tmdb ids input csv file
downloadIMDBIds(input_filename='tmdb-movies-1-to-400.csv', output_filename='imdb-ids-1-to-400.csv')

done with 200 movies
done with 400 movies
done with 600 movies
done with 800 movies
done with 1000 movies
done with 1200 movies
done with 1400 movies
done with 1600 movies
done with 1800 movies
done with 2000 movies
done with 2200 movies
done with 2400 movies
done with 2600 movies
done with 2800 movies
done with 3000 movies
done with 3200 movies
done with 3400 movies
done with 3600 movies
done with 3800 movies
done with 4000 movies
done with 4200 movies
done with 4400 movies
done with 4600 movies
done with 4800 movies
done with 5000 movies
done with 5200 movies
done with 5400 movies
done with 5600 movies
done with 5800 movies
done with 6000 movies
done with 6200 movies
done with 6400 movies
done with 6600 movies
done with 6800 movies
done with 7000 movies
done with 7200 movies
done with 7400 movies
done with 7600 movies
done with 7800 movies
done with 8000 movies


In [7]:
# PART 3: STANDALONE THAT TAKES IN IMDB IDs and gets IMDB features
'''
Make sure you have IMDB installed.
- Go to: http://imdbpy.sourceforge.net/
- Download and unzip, then cd into it and make sure there is a setup.py file
- Run python setup.py install
- You're done! It's globally installed.
'''
import imdb
import pandas as pd
import csv
import requests
import numpy as np

def getIMDBFeatures(input_filename, output_filename, start, increment):

    # Note: This cannot be terminated via the stop button (interrupt the kernel), 
    # got to restart the kernel (use rewind button) :(
    
    ia = imdb.IMDb()
    df = pd.read_csv(input_filename)
    # Download increment movies at a time
    df = df[start:start+increment]
    
    imids = np.array(df['imdb_id'])

    with open(output_filename + '-' + str(start), 'w') as csvfile:
        # Grab these features from IMDB
        fieldnames = ['imdb_id', 'director', 'imdb_votes', 'certificate', 'num_stunts', 'num_fx']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()

        count = 0
        for imid in imids:
            count += 1
            # Tries twice because sometimes it fails
            for i in range(2):
                try:
                    movie = ia.get_movie(str(int(imid)))
                    director = movie['director'][0]
                    imdb_votes = movie['votes']
                    certificate = movie['certificates'][-2].split(':')[1]
                    num_stunts = len(movie['stunt performer'])
                    num_fx = len(movie['special effects department'])
                    row = {'imdb_id': imid, 'director': director, 'imdb_votes': imdb_votes, 'certificate': certificate, 
                          'num_stunts': num_stunts, 'num_fx': num_fx}
                    writer.writerow(row)
                    break
                except:    
                    pass
            if count % 100 == 0:
                print 'Done with %d movies' % count
    print 'Done with page %d' % ((start%increment) + 1)


In [ ]:
### Run Part 3: Get imdb features from imdb ids

# NOTE: This downloads 500 movies at a time and stores each in a different file.

import pandas as pd
df = pd.read_csv('imdb-ids-1-to-400.csv')
N = df.shape[0]
increment = 500 # Work on 500 movies at a time
end_page = N/increment

##############################################################
# NOTE: If you are done with page 2 (1000 movies), then change this to 2 the next time you start
start_page = 2
##############################################################

starts = [] 
for i in range(start_page,end_page): # default starts: [500,1000,1500,2000,2500,..,7500]
    starts.append((i+1)*increment)

for start in starts: 
    getIMDBFeatures(input_filename='imdb-ids-1-to-400.csv', output_filename='imdb-features-1-to-400.csv', 
                    start=start, increment=increment)
